In [74]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
%matplotlib inline

# get database as panda dataframe
df = pd.read_csv('WFPVAM_FoodPrices_version1.csv')

In [75]:
# for a more intuitive access to the database
COUNTRY = 'adm0_name'
REGION = 'adm1_name'
CITY = 'mkt_name'
PROD = 'cm_name'
CURR = 'cur_name'
SELLER = 'pt_name'
UNIT = 'um_name'
MONTH = 'mp_month'
YEAR = 'mp_year'
DATE = 'date'
PRICE = 'mp_price'
SOURCE = 'mp_commoditysource'

# gets all products as keys with according units as values #
dict = {}
for (product, unit), df_group in df.groupby(["cm_name", "um_name"]):
    if not product in dict:
        dict[product] = [unit]
    else:
        dict[product].append(unit)

# comprehends created dict to include only products with inconsistent unit measures #
dict2 = {key: value for (key, value) 
         in dict.items() if len(value) > 1}

# number of products with inconsistent (more than one) unit measures
print(len(dict2))


# returns all unit measures used in database
print(df.eval(UNIT).unique())



96
['KG' 'Day' 'Unit' 'L' 'USD/LCU' '11.5 KG' 'Pound' 'Cuartilla' '90 KG' 'MT'
 'Libra' 'Gallon' '380 G' 'Month' '85 G' '500 ML' 'Course' '45 KG' 'Head'
 '100 KG' '500 G' '50 KG' '91 KG' 'Bunch' '100 Tubers' '650 G' '750 ML'
 '115 G' 'Marmite' '350 G' '385 G' '1.8 KG' '2 KG' '400 G' '10 pcs' '150 G'
 '5 KG' '30 pcs' '5 L' '160 G' '200 G' '12.5 KG' 'Loaf' 'Dozen' 'Sack'
 '1.5 KG' '185 G' '10 KG' 'Packet' '750 G' '1.5 L' '12 KG' '18 KG' '25 KG'
 '3 L' 'Cubic meter' '3 KG' '3.5 KG' '168 G']


In [77]:

# assuming:
# 1 Libra = 0.329 kg
# 1 MT (Metric Ton) = 1000 KG
# 1 L Palm Oil = 0.91298 KG
UNIT_PRICE_CONVERTER = {'1.5 KG': (1.5, 'KG'), '5 KG': (5.0, 'KG'), '1.8 KG': (1.8, 'KG'), 
                        '2 KG': (2.0, 'KG'), '12.5 KG': (12.5, 'KG'), '10 KG': (10.0, 'KG'),
                        '60 KG': (60.0, 'KG'), '18 KG': (18.0, 'KG'), '25 KG': (25.0, 'KG'),
                        '3 KG': (3.0, 'KG'), '3.5 KG': (3.5, 'KG'), '12 KG': (12.0, 'KG'),
                        '500 G': (0.5, 'KG'), '125 G': (0.125, 'KG'), '90 KG': (90.0, 'KG'), 
                        'Pound': (0.45359237, 'KG'), '380 G': (0.380, 'KG'), '85 G': (0.085, 'KG'),
                        '45 KG': (45.0, 'KG'), '100 KG': (100.0, 'KG'), '50 KG': (50.0, 'KG'), 
                        '91 KG': (91.0, 'KG'), '650 G': (0.650, 'KG'), '115 G': (0.115, 'KG'),
                        '350 G': (0.350, 'KG'), '385 G': (0.385, 'KG'), '11.5 KG': (11.5, 'KG'),
                        '400 G': (0.400, 'KG'), '150 G': (0.150, 'KG'), '160 G': (0.160, 'KG'), 
                        '200 G': (0.200, 'KG'), '185 G': (0.185, 'KG'), '750 G': (0.750, 'KG'),
                        '168 G': (0.168, 'KG'), 'Libra': (0.329, 'KG'), 'MT': (1000.0, 'KG'),
                        'Gallon': (3.78541178, 'L'), '500 ML': (0.500, 'L'), '750 ML': (0.750, 'L'),
                        '1.5 L': (1.5, 'L'), '3 L': (3.0, 'L'), 'Cubic Meter': (1000.0, 'L'),
                        '5 L': (5.0, 'L'), 
                        '10 pcs': (10.0, 'pcs'), '30 pcs': (30.0, 'pcs'), 'Dozen': (12.0, "pcs")}
                        
                        

CONV_PRICE = 0
CONV_UNIT = 1

def get_values_column(df, column_name, value):
    """
    df = dataset
    column_name = kolom waarin je wil zoeken
    value = waarde die je uit de kolom wil.

    return nieuwe dataset met alle rijen die alleen de value in de gegeven kolommen bevatten.
    """
    return df.loc[df[column_name] == value]

def norm_price_unit(row, Col):
    """

    """
    if Col:
        try:
            return UNIT_PRICE_CONVERTER[row.get(UNIT)][Col]
        except:
            return row.get(UNIT)
    else:
        try:
            return row.get(PRICE) / UNIT_PRICE_CONVERTER[row.get(UNIT)][Col]
        except:
            return row.get(PRICE)

def normalize(df):
    
    df[PRICE] = df.apply(lambda row: norm_price_unit(row, CONV_PRICE), axis = 1)
    df[UNIT] = df.apply(lambda row: norm_price_unit(row, CONV_UNIT), axis = 1)


normalize(df)

dict = {}
for (product, unit), df_group in df.groupby(["cm_name", "um_name"]):
    if not product in dict:
        dict[product] = [unit]
    else:
        dict[product].append(unit)

dict2 = {key: value for (key, value)
         in dict.items() if len(value) > 1}

print(len(dict2))
print(df.eval(UNIT).unique())



30
['KG' 'Day' 'Unit' 'L' 'USD/LCU' 'Cuartilla' 'Month' 'Course' 'Head'
 'Bunch' '100 Tubers' 'Marmite' 'pcs' 'Loaf' 'Sack' 'Packet' 'Cubic meter']


In [78]:
# get products with multiple unit measures in one city
A = df.groupby([CITY, PROD])
print({len(tmp_df.eval(UNIT).unique()) for _, tmp_df in A})
[(group,tmp_df.eval(UNIT).unique()) for group, tmp_df in A if len(tmp_df.eval(UNIT).unique()) > 1]



# to do:
# normalize measures:
# - kg to Liters, pieces to KG etc if possible
# - otherwise: count frequency of used measure:
        # if freq is low: delete data
        # if freg is high: add different name for product with that measure (eg: 'bananas (kg)' and 'bananas (bunch)')


{1, 2, 3}


[(('Banjul', 'Coffee (instant)'), array(['KG', 'Unit'], dtype=object)),
 (('Basse Santa su', 'Coffee (instant)'), array(['KG', 'Unit'], dtype=object)),
 (('Biu', 'Oil (palm)'), array(['KG', 'L'], dtype=object)),
 (('Brikama', 'Coffee (instant)'), array(['KG', 'Unit'], dtype=object)),
 (('Damaturu', 'Oil (palm)'), array(['KG', 'L'], dtype=object)),
 (('Gaalkacyo', 'Fuel (diesel)'), array(['KG', 'L'], dtype=object)),
 (('Garowe', 'Fuel (diesel)'), array(['KG', 'L'], dtype=object)),
 (('Medellin', 'Milk (pasteurized)'), array(['L', 'KG'], dtype=object)),
 (('Mubi', 'Oil (palm)'), array(['KG', 'L'], dtype=object)),
 (('National Average', 'Eggs'), array(['KG', 'Unit', 'pcs'], dtype=object)),
 (('National Average', 'Oil (olive)'), array(['KG', 'L'], dtype=object)),
 (('National Average', 'Oil (sunflower)'), array(['KG', 'L'], dtype=object)),
 (('Serrekunda', 'Coffee (instant)'), array(['KG', 'Unit'], dtype=object)),
 (('Soma', 'Coffee (instant)'), array(['KG', 'Unit'], dtype=object))]

In [95]:


is_eggs = df[PROD] == 'Eggs'
print(df[is_eggs][UNIT].value_counts())

is_coffee = df[PROD] == 'Coffee (instant)'
print(df[is_coffee][UNIT].value_counts())


is_oil = df[PROD] == 'Oil (palm)'
print(df[is_oil][UNIT].value_counts())


Unit    5177
pcs     3835
KG       749
Name: um_name, dtype: int64
Unit    155
KG      132
Name: um_name, dtype: int64
L     7174
KG    2199
Name: um_name, dtype: int64


In [ ]:
# 1 L Palm Oil = 0.91298 KG
oil_price_to_KG = lambda x: x * 1.09531

is_palm_oil = df[PROD] == 'Oil (palm)'
is_in_Liters = df[UNIT] == 'L'

df[is_palm_oil & is_in_Liters][PRICE].apply(oil_price_to_KG)

#df[is_palm_oil & is_in_Liters][PRICE] = df[is_palm_oil & is_in_Liters][PRICE].apply(Oil_to_KG)
#df[is_palm_oil & is_in_Liters][UNIT] = "KG"
#df[PROD] = 'Oil (palm)'